# 0. 前置

In [4]:
%run"MLP_lib.ipynb"
import librosa, os
import numpy as np
import tensorflow as tf
import pandas as pd
sess = tf.Session()

#  
#  

# 1.資料處理

## My path

In [5]:
path0 = "/Users/chtsou/Desktop/統研/論文相關/database/0922meeting用3"

## 1.1 Read data

In [6]:
data = connect_data_folder(path0, ["一", "丁", "七", "刀", "八"])
print(data)
print("  ")
print("Dims of x_input:", data[0].shape)
print("Dims of y_input:", data[1].shape)

(array([[ -4.82142000e+01,  -6.87033400e+00,   4.56417500e+01, ...,
         -7.94552900e-01,  -1.04740800e+00,   3.74651000e+00],
       [ -2.95971800e+01,   1.09335200e+01,   4.33798300e+01, ...,
          2.70893100e+00,   2.90836500e+00,   8.69913000e-01],
       [ -5.52936300e+01,  -3.83949800e+00,   4.83102200e+01, ...,
         -7.92458100e-01,  -9.50152400e-01,   4.61278000e-01],
       ..., 
       [ -3.80405000e+01,   2.41489700e+01,   5.35279800e+00, ...,
          2.14572200e-01,  -4.45081300e-01,  -2.28587600e+00],
       [ -2.75461700e+01,   1.10000200e+01,  -3.40320000e+00, ...,
          1.09544300e+00,   1.31733100e+00,   2.45061100e+00],
       [ -2.62666800e+01,   2.56865200e+01,   3.01310200e+00, ...,
          7.99602700e-01,  -5.46483700e-02,   2.98498700e+00]]), array([[ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.

## 1.2 Splits train & test
### 訓練464筆，測試52筆

In [7]:
nrow = len(data[0][:, 1])
test_x, test_y = data[0][range(3, nrow, 10)], data[1][range(3, nrow, 10)]
train_x, train_y = np.delete(data[0], range(3, nrow, 10), 0), np.delete(data[1], range(3, nrow, 10), 0)

train_data = train_x, train_y
test_data = test_x, test_y

print("Size of train data:", train_x.shape[0])
print("Size of train data:", test_x.shape[0])

Size of train data: 419
Size of train data: 47


## 1.3 Spliting train&test data by frame 

In [8]:
train_by_frame = frame_split22(train_data, 25)
print("Dims of ith sp_frame_x:", train_by_frame[0].shape)

print("Dims of ith sp_frame_y:", train_by_frame[1].shape)

print("-------------------------------------")
test_by_frame = frame_split22(test_data, 25)
print("Dims of ith sp_frame_x:", test_by_frame[0].shape)

print("Dims of ith sp_frame_y:", test_by_frame[1].shape)

Dims of ith sp_frame_x: (25, 419, 39)
Dims of ith sp_frame_y: (419, 5)
-------------------------------------
Dims of ith sp_frame_x: (25, 47, 39)
Dims of ith sp_frame_y: (47, 5)


## 1.4 Scaling data by train_mean & train_sd

In [12]:
scale_tup = get_mean_and_sd(train_data[0])
train_x_std = scale_by_train(train_by_frame[0], scale_tup[0], scale_tup[1])
test_x_std = scale_by_train(test_by_frame[0], scale_tup[0], scale_tup[1])

#  
#  
#  

# 2. 搭建框架

## 2.1 設定各層神經元

In [13]:
n_in = 39
n_h = 100
n_out = 5
n_fr = 25

## 2.2 定義layer

In [14]:
def add_layer(inputs, in_size, out_size, activation_function=None):
    # add one more layer and return the output of this layer
    with tf.name_scope('layer'):
        with tf.name_scope('weights'):
#             Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            Weights = tf.Variable(tf.random_normal([in_size, out_size], mean=0., stddev=0.3))
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b') # 
#             biases = tf.Variable(tf.zeros([1, out_size]) + 0.03)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
        return outputs

In [15]:
hidden_list = list(np.zeros(n_fr))
prediction_list = list(np.zeros(n_fr))
xs = tf.placeholder(tf.float32, [None, n_in])
ys = tf.placeholder(tf.float32, [None, n_out])
for i in range(n_fr):
    hidden_list[i] = add_layer(xs, n_in, n_h, activation_function=tf.nn.tanh)
    prediction_list[i] = add_layer(hidden_list[i], n_h, n_out, activation_function=tf.nn.softmax)

In [16]:
def compute_accuracy(v_xs, v_ys, prediction):
    y_pre = sess.run(prediction, feed_dict={xs: v_xs})  
    correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    result = sess.run(accuracy, feed_dict={xs: v_xs, ys: v_ys})
    return result

#  
#   
#   

In [17]:
def output_pr_frame(v_xs, prediction):
    y_pre = sess.run(prediction, feed_dict={xs: v_xs})  
#     correct_prediction = tf.equal(tf.argmax(y_pre, 1), tf.argmax(v_ys, 1))
#     y = sess.run(correct_prediction)
    return y_pre

In [18]:
# test_data: tuple of test data
# i_fr: 分音框數
# pr_array每列和為25
def test_accuracy(test_x_by_frame, test_y, n_fr, prediction):
    sess = tf.Session()
    
    input_x = test_x_by_frame[0]
    input_y = test_y
    y_pre = output_pr_frame(input_x, prediction[0])
    y_pre_shape = y_pre.shape        # 容器大小
    pr_array = tf.zeros(y_pre_shape) # 容器
    
    for n_fr in range(n_fr):
        input_x = test_x_by_frame[n_fr]
#         sess.run(prediction, feed_dict={xs: v_xs})
        pr_array += output_pr_frame(input_x, prediction[n_fr])
        
    correct_prediction = tf.equal(tf.argmax(pr_array, 1), tf.argmax(input_y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    result = sess.run(accuracy)
    return result

# 3. 訓練模型

## 3.1 初始化

In [19]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

## 3.2 開始逐個音框訓練模型，正確率夠大就提早停止

In [280]:
batch_ys = train_y
input_y = train_y
for fr_index in range(25):
    batch_xs = train_by_frame[0][fr_index, :, :]
    input_x = train_by_frame[0][fr_index, :, :]
    
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction_list[fr_index]), reduction_indices=[1]))
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
    
    print("---frame", fr_index + 1)
    for j in range(1000):
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
        if j % 100 == 0:
            c = compute_accuracy(input_x, input_y, prediction_list[fr_index])
            print(c)
        if c > 0.99:
            print(c)
            break

---frame 1
0.696897
0.990453
0.990453
---frame 2
0.763723
0.995227
0.995227
---frame 3
0.821002
0.997613
0.997613
---frame 4
0.830549
0.988067
0.995227
0.995227
---frame 5
0.778043
0.988067
0.995227
0.995227
---frame 6
0.825776
0.995227
0.995227
---frame 7
0.763723
0.995227
0.995227
---frame 8
0.75895
0.990453
0.990453
---frame 9
0.789976
0.98568
0.997613
0.997613
---frame 10
0.804296
0.990453
0.990453
---frame 11
0.832936
0.99284
0.99284
---frame 12
0.809069
0.988067
0.995227
0.995227
---frame 13
0.809069
0.99284
0.99284
---frame 14
0.828162
0.988067
0.997613
0.997613
---frame 15
0.789976
0.995227
0.995227
---frame 16
0.813842
0.99284
0.99284
---frame 17
0.842482
0.990453
0.990453
---frame 18
0.785203
0.988067
0.997613
0.997613
---frame 19
0.809069
0.990453
0.990453
---frame 20
0.799523
0.990453
0.990453
---frame 21
0.77327
0.98568
1.0
1.0
---frame 22
0.768496
0.988067
1.0
1.0
---frame 23
0.768496
0.997613
0.997613
---frame 24
0.761337
0.995227
0.995227
---frame 25
0.749403
0.990453
0

In [281]:
print(test_accuracy(train_by_frame[0], train_by_frame[1], 25, prediction_list))
print(test_accuracy(test_by_frame[0], test_by_frame[1], 25, prediction_list))

1.0
0.87234


In [327]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# 標準化訓練

In [20]:
batch_ys = train_y
input_y = train_y
for fr_index in range(25):
    batch_xs = train_x_std[fr_index, :, :]
    input_x = train_x_std[fr_index, :, :]
    
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction_list[fr_index]), reduction_indices=[1]))
    train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)
    
    print("---frame", fr_index + 1)
    for j in range(3000):
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
        if j % 1000 == 0:
            c = compute_accuracy(input_x, input_y, prediction_list[fr_index])
            print(c)
        if c > 0.99:
            print(c)
            break

---frame 1
0.279236
1.0
1.0
---frame 2
0.231504
1.0
1.0
---frame 3
0.279236
1.0
1.0
---frame 4
0.236277
1.0
1.0
---frame 5
0.229117
1.0
1.0
---frame 6
0.252983
1.0
1.0
---frame 7
0.200477
1.0
1.0
---frame 8
0.264916
1.0
1.0
---frame 9
0.195704
0.997613
0.997613
---frame 10
0.229117
1.0
1.0
---frame 11
0.231504
1.0
1.0
---frame 12
0.171838
1.0
1.0
---frame 13
0.169451
0.997613
0.997613
---frame 14
0.167064
0.997613
0.997613
---frame 15
0.176611
0.997613
0.997613
---frame 16
0.295943
1.0
1.0
---frame 17
0.231504
0.990453
0.990453
---frame 18
0.22673
0.99284
0.99284
---frame 19
0.229117
0.997613
0.997613
---frame 20
0.264916
0.995227
0.995227
---frame 21
0.193317
0.988067
1.0
1.0
---frame 22
0.23389
0.995227
0.995227
---frame 23
0.186158
0.995227
0.995227
---frame 24
0.23389
0.995227
0.995227
---frame 25
0.210024
0.99284
0.99284


In [21]:
print(test_accuracy(train_x_std, train_by_frame[1], 25, prediction_list))
print(test_accuracy(test_x_std, test_by_frame[1], 25, prediction_list))

1.0
0.914894


# 隨機抽取

In [27]:
sample_size = train_y.shape[0]

input_y = train_y
batch_size = 30

for fr_index in range(25):
    input_x = train_x_std2[fr_index, :, :]
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction_list[fr_index]), reduction_indices=[1]))
    train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
    print("---frame", fr_index + 1)
    for j in range(3000): 
        bat_i = np.random.choice(range(sample_size), batch_size)
    
        batch_xs = train_x_std2[fr_index, bat_i, :]
        batch_ys = train_y[bat_i, :]
        sess.run(train_step, feed_dict={xs: batch_xs, ys: batch_ys})
        if j % 1000 == 0:
            c = compute_accuracy(input_x, input_y, prediction_list[fr_index])
            print(c)
        if c > 0.95:
            print(c)
            break

---frame 1
1.0
1.0
---frame 2
1.0
1.0
---frame 3
1.0
1.0
---frame 4
1.0
1.0
---frame 5
1.0
1.0
---frame 6
1.0
1.0
---frame 7
1.0
1.0
---frame 8
1.0
1.0
---frame 9
0.997613
0.997613
---frame 10
1.0
1.0
---frame 11
1.0
1.0
---frame 12
1.0
1.0
---frame 13
1.0
1.0
---frame 14
1.0
1.0
---frame 15
1.0
1.0
---frame 16
1.0
1.0
---frame 17
1.0
1.0
---frame 18
1.0
1.0
---frame 19
1.0
1.0
---frame 20
1.0
1.0
---frame 21
1.0
1.0
---frame 22
1.0
1.0
---frame 23
1.0
1.0
---frame 24
0.997613
0.997613
---frame 25
0.995227
0.995227


In [28]:
print(test_accuracy(train_x_std2, train_by_frame[1], 25, prediction_list))
print(test_accuracy(test_x_std2, test_by_frame[1], 25, prediction_list))

1.0
0.914894


In [5]:
1-(4/47)

0.9148936170212766